In [4]:
import pandas as pd
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [5]:
grain = pd.read_csv('https://byui-cse.github.io/cse450-course/ice/grain/data/grain-training.csv')

In [6]:
grain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3048 entries, 0 to 3047
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Area               3048 non-null   int64  
 1   Perimeter          3048 non-null   float64
 2   Major_Axis_Length  3048 non-null   float64
 3   Minor_Axis_Length  3048 non-null   float64
 4   Eccentricity       3048 non-null   float64
 5   Convex_Area        3048 non-null   int64  
 6   Extent             3048 non-null   float64
 7   Class              3048 non-null   object 
dtypes: float64(5), int64(2), object(1)
memory usage: 190.6+ KB


In [7]:
grain.head()

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent,Class
0,11366,423.114990,171.906647,85.579300,0.867278,11599,0.611404,Osmancik
1,16523,531.892029,224.995422,94.417702,0.907689,16911,0.577041,Cammeo
2,11088,418.208008,172.027420,82.935669,0.876112,11284,0.624993,Osmancik
3,14528,475.447998,192.198563,97.417427,0.862029,14795,0.629490,Cammeo
4,8990,389.377014,157.749603,73.919182,0.883418,9297,0.625261,Osmancik


In [8]:
grain.describe()

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent
count,3048.000000,3048.000000,3048.000000,3048.000000,3048.000000,3048.000000,3048.000000
mean,12650.463583,453.910420,188.634550,86.259638,0.886847,12933.628937,0.663072
std,1723.577307,35.418271,17.360515,5.736319,0.020829,1767.781362,0.077284
min,7551.000000,359.100006,145.264465,59.532406,0.777233,7723.000000,0.497413
25%,11357.750000,425.958992,174.235886,82.642729,0.872278,11608.750000,0.599417
50%,12415.000000,448.593506,185.520943,86.279121,0.889010,12681.500000,0.648185
75%,13928.250000,483.222244,203.312111,90.120306,0.902608,14268.500000,0.728501
max,18313.000000,548.445984,238.435089,107.542450,0.948007,18724.000000,0.861050


In [9]:
# b = grain.total.unique()
# print(b)

In [10]:
grain['Class'] = (grain['Class'] == 'Osmancik').astype(int)

In [11]:
grain.head()

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent,Class
0,11366,423.114990,171.906647,85.579300,0.867278,11599,0.611404,1
1,16523,531.892029,224.995422,94.417702,0.907689,16911,0.577041,0
2,11088,418.208008,172.027420,82.935669,0.876112,11284,0.624993,1
3,14528,475.447998,192.198563,97.417427,0.862029,14795,0.629490,0
4,8990,389.377014,157.749603,73.919182,0.883418,9297,0.625261,1


In [12]:
grain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3048 entries, 0 to 3047
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Area               3048 non-null   int64  
 1   Perimeter          3048 non-null   float64
 2   Major_Axis_Length  3048 non-null   float64
 3   Minor_Axis_Length  3048 non-null   float64
 4   Eccentricity       3048 non-null   float64
 5   Convex_Area        3048 non-null   int64  
 6   Extent             3048 non-null   float64
 7   Class              3048 non-null   int32  
dtypes: float64(5), int32(1), int64(2)
memory usage: 178.7 KB


In [13]:
X = grain.drop("Class", axis=1)
y = grain.Class

In [14]:
X

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent
0,11366,423.114990,171.906647,85.579300,0.867278,11599,0.611404
1,16523,531.892029,224.995422,94.417702,0.907689,16911,0.577041
2,11088,418.208008,172.027420,82.935669,0.876112,11284,0.624993
3,14528,475.447998,192.198563,97.417427,0.862029,14795,0.629490
4,8990,389.377014,157.749603,73.919182,0.883418,9297,0.625261
...,...,...,...,...,...,...,...
3043,14078,478.470001,203.645462,88.560310,0.900491,14280,0.744395
3044,17246,540.541992,225.988861,98.573151,0.899857,17704,0.573929
3045,11070,419.403015,173.575043,82.154213,0.880898,11266,0.600586
3046,11747,452.127014,194.494858,78.744461,0.914376,11935,0.542637


In [15]:
y

0       1
1       0
2       1
3       0
4       1
       ..
3043    0
3044    0
3045    1
3046    0
3047    1
Name: Class, Length: 3048, dtype: int32

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
# Initialize and train the model
model = LogisticRegressionCV()
model.fit(X_train, y_train)

# Make predictions and evaluate the model
predictions = model.predict(X_test)
result = mean_squared_error(y_test, predictions, squared=False)

In [32]:
predictions

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,

In [22]:
result = mean_squared_error(y_test, predictions, squared=False)
result

0.29476303347204064

In [23]:
r2 = r2_score(y_test,predictions)
r2

0.6443696443696444

In [24]:
accuracy_score(y_test, predictions)

0.9131147540983606

In [25]:
holdout = pd.read_csv('https://byui-cse.github.io/cse450-course/ice/grain/data/grain-holdout.csv')

In [26]:
holdout.head()

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent
0,12899,472.263000,205.520554,80.433365,0.920236,13107,0.585919
1,11693,424.503998,176.024063,84.991318,0.875709,11842,0.606735
2,11329,419.404999,173.390457,84.345825,0.873708,11560,0.803247
3,12278,453.692993,184.050156,86.660088,0.882213,12601,0.629835
4,11231,427.666992,183.587097,78.861732,0.903038,11427,0.721184


In [28]:
predict_holdout = model.predict(holdout)
predict_holdout

array([0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,

In [29]:
converted_predictions = np.where(predict_holdout < 0.5, "Cammeo", "Osmancik")

In [30]:
converted_predictions

array(['Cammeo', 'Osmancik', 'Osmancik', 'Osmancik', 'Osmancik', 'Cammeo',
       'Cammeo', 'Cammeo', 'Osmancik', 'Osmancik', 'Osmancik', 'Osmancik',
       'Cammeo', 'Osmancik', 'Cammeo', 'Osmancik', 'Osmancik', 'Osmancik',
       'Osmancik', 'Cammeo', 'Osmancik', 'Cammeo', 'Osmancik', 'Osmancik',
       'Cammeo', 'Cammeo', 'Cammeo', 'Cammeo', 'Cammeo', 'Cammeo',
       'Cammeo', 'Cammeo', 'Cammeo', 'Cammeo', 'Cammeo', 'Cammeo',
       'Cammeo', 'Cammeo', 'Osmancik', 'Cammeo', 'Osmancik', 'Osmancik',
       'Cammeo', 'Osmancik', 'Osmancik', 'Cammeo', 'Osmancik', 'Osmancik',
       'Osmancik', 'Cammeo', 'Osmancik', 'Osmancik', 'Osmancik', 'Cammeo',
       'Cammeo', 'Osmancik', 'Osmancik', 'Osmancik', 'Cammeo', 'Cammeo',
       'Cammeo', 'Cammeo', 'Osmancik', 'Cammeo', 'Cammeo', 'Cammeo',
       'Cammeo', 'Osmancik', 'Osmancik', 'Osmancik', 'Osmancik', 'Cammeo',
       'Cammeo', 'Cammeo', 'Osmancik', 'Cammeo', 'Cammeo', 'Osmancik',
       'Cammeo', 'Cammeo', 'Osmancik', 'Osmancik', 'Cam

In [ ]:
output_file = "duane_siu-LogisticRegression-ice-grain-predictions.csv"
predict_holdout = pd.DataFrame(converted_predictions, columns=['predictions'])
predict_holdout.to_csv(output_file, index = False)
print(f"Dataset saved to {output_file}. You can download it from here.")

Dataset saved to duane_siu-ice-grain-predictions.csv. You can download it from here.
